In [1]:
import numpy as np
import skimage.io as io
import random
import os
import albumentations as A
import cv2
import pickle
import torch
from pycocotools.coco import COCO
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensorV2
import glob
import time
from random import shuffle

In [2]:
torch.cuda.device_count()
device = torch.device('cuda:7')
torch.cuda.empty_cache()

In [3]:
mode = 'val'

load_folder = '/shared/data/COCOdataset2017/images/{}'.format(mode)
save_folder = '/shared/suhyeon/GCCdataset'
mask_folder = '{}/masks/{}'.format(save_folder, mode)
ann_file = '/shared/data/COCOdataset2017/annotations/instances_{}.json'.format(mode)
coco = COCO(ann_file)

batch_size = 1
input_size = (256, 256)
SMOOTH = 1e-6

loading annotations into memory...
Done (t=0.71s)
creating index...
index created!


In [4]:
with open('./validAnns_val.pkl', 'rb') as f:
    validAnns = pickle.load(f)

with open('./top5_dict_val.pkl', 'rb') as f:
    top5_dict = pickle.load(f)
    
# def getImage(self, imageObj, img_folder, img_size):
#         # Read and normalize an image
#         train_img = cv2.imread(img_folder + '/' + imageObj['file_name'], cv2.IMREAD_COLOR)
#         train_img = cv2.cvtColor(train_img, cv2.COLOR_BGR2RGB)
#         train_img = cv2.resize(train_img, img_size) # (W,H)
#         return train_img


In [5]:
len(top5_dict.items())

6679

In [7]:
print(len(validAnns))

6679


In [8]:
# 1) load all images, masks
# torch XXXX opencv operation needed

images = {}
masks = {}

start = time.time()

# images folder glob -> transform (256,256)
for i, (catId, imgId, annId) in enumerate(validAnns):
    image = cv2.imread(load_folder + '/' + str(imgId).zfill(12) + '.jpg', cv2.IMREAD_COLOR)
    image = cv2.resize(image, input_size)
    images[imgId] = image
    # mask = trans(mask).type(torch.bool).to(device)
    
    mask = cv2.imread(mask_folder + '/' + str(annId) + '.png', cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, input_size)
    # mask = trans(mask).type(torch.bool).to(device)
    masks[annId] = mask
    
end = time.time()

print(f"data loaded: {end - start:.5f} sec")

data loaded: 51.46858 sec


In [9]:
print(len(images), len(masks))

3669 6679


In [10]:
# masks[6000].shape

In [10]:
# 2) not load images, just get images from file paths
### For visualizing the outputs ###
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

kernel = np.ones((3, 3), np.uint8)
SMOOTH = 1e-6
total = 0

In [37]:
from collections import defaultdict
print('# class:', len(coco.getCatIds()))
cnt_dict = defaultdict(int)
top5_list = list(top5_dict.items())
shuffle(top5_list)

# class: 80


In [21]:
# top5_list[0]

((21, 235857, 72894),
 [(357081, 71340, tensor(0.4718, device='cuda:0')),
  (334555, 1819582, tensor(0.4337, device='cuda:0')),
  (219440, 70911, tensor(0.4076, device='cuda:0')),
  (535094, 2177055, tensor(0.3930, device='cuda:0')),
  (244411, 72392, tensor(0.3887, device='cuda:0'))])

In [28]:
len(top5_list)

6679

In [38]:
cnt_dict

defaultdict(int, {})

In [39]:
for i, ((catId_1, imgId_1, annId_1), cands) in enumerate(top5_list):
    # start = time.time()
    # cat_id = coco.loadAnns(annId_1)[0]['category_id']
    # print(catId_1, cnt_dict[catId_1], annId_1, cands)

    img1 = images[imgId_1]
    mask1 = masks[annId_1]
    
    # bg1 = cv2.bitwise_and(img1, img1, mask = mask1) 
    # invert1 = cv2.bitwise_not(mask1)//255
    # fg1 = cv2.bitwise_and(img1, img1, mask = invert1)
    
    # cv2.imwrite(save_folder + '/val/src/' + str(annId_1).zfill(12) + '_bg' + '.png', bg1)
    # cv2.imwrite(save_folder + '/val/src/' + str(annId_1).zfill(12) + '_fg' + '.png', fg1)
    # cv2.imwrite(save_folder + '/val/src/' + str(annId_1).zfill(12) + '_gt' + '.png', img1)
    # cv2.imwrite(save_folder + '/val/src/' + str(annId_1).zfill(12) + '_mask' + '.png', mask1)
    
    if cnt_dict[catId_1] < 5:
        imgId_2, annId_2, _ = cands[np.random.randint(5)] #top5
        # for j, (imgId_2, annId_2, _) in enumerate(cands):
        if imgId_2 != 0 and annId_2 != 0:
            cnt_dict[catId_1] += 1
            
            print(cat_id, imgId_1, annId_1, imgId_2, annId_2, cnt_dict[catId_1])

            img2 = images[imgId_2]
            mask2 = masks[annId_2]

#             bg2 = cv2.bitwise_and(img2, img2, mask = mask2) 
#             invert2 = cv2.bitwise_not(mask2)//255
#             fg2 = cv2.bitwise_and(img2, img2, mask = invert2)

#             com = cv2.copyTo(fg2, mask2, img1.copy())

#             # cv2.imwrite(save_folder + '/val/tgt/' + str(annId_1).zfill(12) + '_tgt_' + str(annId_2).zfill(12) + '_bg' + '.png', bg2)
#             # cv2.imwrite(save_folder + '/val/tgt/' + str(annId_1).zfill(12) + '_tgt_' + str(annId_2).zfill(12) + '_com' + '.png', com)
#             # cv2.imwrite(save_folder + '/val/tgt/' + str(annId_1).zfill(12) + '_tgt_' + str(annId_2).zfill(12) + '_fg' + '.png', fg2)
#             # cv2.imwrite(save_folder + '/val/tgt/' + str(annId_1).zfill(12) + '_tgt_' + str(annId_2).zfill(12) + '_gt' + '.png', img2)
#             # cv2.imwrite(save_folder + '/val/tgt/' + str(annId_1).zfill(12) + '_tgt_' + str(annId_2).zfill(12) + '_mask' + '.png', mask2)
    
#     end = time.time()
#     elapsed = end - start
#     total += elapsed
    
    # print("[{}/{}]".format(i, len(top5_dict)), f"average time: {total/(i+SMOOTH):.5f} sec", end='\r')

82 350019 1186373 112626 1186360 1
82 117719 904400117719 79651 89208 1
82 200252 316081 40757 1612234 1
82 534664 1444560 261318 1184301 2
82 2685 1715694 388927 1235070 1
82 462643 1725788 209530 1280088 2
82 442836 444724 509451 1214639 3
82 512194 1157219 550426 1152992 1
82 210789 460314 429761 1204085 4
82 317999 2154303 10764 471070 5
82 508602 131723 167122 134420 1
82 147745 398974 401862 398534 1
82 49761 901600049761 315257 39785 1
82 376264 1144753 536038 1650485 1
82 549220 639100 481573 644658 1
82 203488 1396085 306437 1811169 1
82 35279 1139079 375278 1145356 2
82 172330 145333 338718 354081 2
82 334977 1639255 462643 322436 1
82 571893 1641786 802 1123710 1
82 95899 270627 79408 270680 1
82 565778 174719 275727 173188 1
82 237864 580034 24919 583585 1
82 148620 1097814 232684 1099085 1
82 190140 1375512 532575 177574 1
82 330790 580872 28449 582603 2
82 535253 1041540 165351 1042872 1
82 366611 8705 22192 3253 1
82 236784 98044 241326 113605 1
82 454750 591083 116589 5

In [13]:
cands

[(160666, 342306, tensor(0.1117, device='cuda:0')),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0)]

In [ ]:
# print(cnt_dict)
    
#     f, ax = plt.subplots(nrows=1, ncols=5, figsize=(8, 8))
#     f.tight_layout()

#     ax[0].imshow(bg.squeeze())
#     ax[0].set_title("bg")
#     ax[0].axis('off')

#     ax[1].imshow(fg.squeeze())
#     ax[1].set_title("fg")
#     ax[1].axis('off')
    
#     ax[2].imshow(mask1.squeeze())
#     ax[2].set_title("mask1")
#     ax[2].axis('off')
    
#     ax[3].imshow(img1.squeeze())
#     ax[3].set_title("img1")
#     ax[3].axis('off')
    
#     ax[4].imshow(invert.squeeze())
#     ax[4].set_title("invert")
#     ax[4].axis('off')

# image save with title 'annId_imgtype_i' (including 0)
# warning: RGB BGR before save
# imgtype: [bg, fg, gt, mask]